In [ ]:
pip install prophet


In [4]:
from google.colab import files
uploaded = files.upload()

Saving retail_sales_dataset.csv to retail_sales_dataset.csv


In [5]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
import seaborn as sns


loading the dataset

In [6]:
df=pd.read_csv('retail_sales_dataset.csv')
df['Date']=pd.to_datetime(df['Date'])
print(df.head())

   Transaction ID       Date Customer ID  Gender  Age Product Category  \
0               1 2023-11-24     CUST001    Male   34           Beauty   
1               2 2023-02-27     CUST002  Female   26         Clothing   
2               3 2023-01-13     CUST003    Male   50      Electronics   
3               4 2023-05-21     CUST004    Male   37         Clothing   
4               5 2023-05-06     CUST005    Male   30           Beauty   

   Quantity  Price per Unit  Total Amount  
0         3              50           150  
1         2             500          1000  
2         1              30            30  
3         1             500           500  
4         2              50           100  


In [ ]:
daily_sales=df.groupby('Date')['Total Amount'].sum().reset_index()
daily_sales.columns=['ds','y']
model=Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    seasonality_mode='multiplicative'
)
model.add_country_holidays(country_name='US')
model.fit(daily_sales)
future=model.make_future_dataframe(periods=90)
forecast=model.predict(future)

DEBUG:cmdstanpy:input tempfile: /tmp/tmp9yiqu04p/mqs2ckn7.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp9yiqu04p/hccap2nh.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=34716', 'data', 'file=/tmp/tmp9yiqu04p/mqs2ckn7.json', 'init=/tmp/tmp9yiqu04p/hccap2nh.json', 'output', 'file=/tmp/tmp9yiqu04p/prophet_modell7usce24/prophet_model-20250722084602.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
08:46:02 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
08:46:03 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


plot the forecast

In [1]:
fig1 = model.plot(forecast)
plt.title('Sales Forecast with Prophet')
plt.xlabel('Date')
plt.ylabel('Total Sales Amount')
plt.show()

NameError: name 'model' is not defined

ploting the components(trend,weekly/yearly seasonality)

In [ ]:
from prophet.plot import plot_plotly,plot_components_plotly
plot_plotly(model,forecast)
plot_components_plotly(model,forecast)
fig2 = model.plot_components(forecast)
plt.show()


CALCULATING THE PERFORMANCE METRICS

In [ ]:
from prophet.diagnostics import cross_validation,performance_metrics
df_cv=cross_validation(model,initial='180 days',period='30 days',horizon='90 days')
df_p=performance_metrics(df_cv)
print(df_p.head())

INFO:prophet:Making 4 forecasts with cutoffs between 2023-07-05 00:00:00 and 2023-10-03 00:00:00


  0%|          | 0/4 [00:00<?, ?it/s]

DEBUG:cmdstanpy:input tempfile: /tmp/tmp9yiqu04p/0miuawlo.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp9yiqu04p/fjmu692f.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=86220', 'data', 'file=/tmp/tmp9yiqu04p/0miuawlo.json', 'init=/tmp/tmp9yiqu04p/fjmu692f.json', 'output', 'file=/tmp/tmp9yiqu04p/prophet_model7xgs6na5/prophet_model-20250722085502.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
08:55:02 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
08:55:02 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp9yiqu04p/96lj9w96.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp9yiqu04p/yc8742aq.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

  horizon           mse         rmse         mae      mape     mdape  \
0  9 days  1.734884e+06  1317.150098  914.781148  2.372320  0.586883   
1 10 days  1.267947e+06  1126.031523  805.956600  2.898737  0.659023   
2 11 days  1.247785e+06  1117.043123  771.885039  3.372075  0.384682   
3 12 days  1.248360e+06  1117.300201  763.650725  3.671469  0.496523   
4 13 days  1.281775e+06  1132.155230  785.349183  3.617263  0.387251   

      smape  coverage  
0  0.773351  0.848485  
1  0.783330  0.878788  
2  0.783297  0.878788  
3  0.794159  0.878788  
4  0.783962  0.878788  
